In [1]:
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from keras.layers import *
import tensorflow as tf 
import numpy as np
import model as model

c:\Users\phong\.conda\envs\tf\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.12.2 when it was built against 1.12.1, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


## Data preparation

In [2]:
# Prepare field data
year_list_train = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020] # Train for 10 years
# Re_list_test = [350, 550, 750, 950]

full_field_data_whole = []
for year in year_list_train:
    filename_field = str(year) + "_east_sea.npy"
    full_field_data_whole.append(np.load(filename_field))

full_field_data_whole_array = np.concatenate(full_field_data_whole, axis = 0)

print(full_field_data_whole_array.shape)

# Normalization
min_val = np.amin(full_field_data_whole_array)
max_val = np.amax(full_field_data_whole_array)
full_field_data_whole_array_norm = (full_field_data_whole_array - min_val)/(max_val - min_val)

# Test
print("Check field data: ", np.amin(full_field_data_whole_array_norm), np.amax(full_field_data_whole_array_norm))

# Create tf.dataset
dataset = tf.data.Dataset.from_tensor_slices(full_field_data_whole_array_norm)
dataset = dataset.shuffle(buffer_size = 2192) 
dataset = dataset.batch(8)
print(dataset)


(17520, 256)
(17520, 128, 128, 1)
Check field data:  0.0 1.0
Check sens data:  0.0009686941213763089 0.9253287962832636
<BatchDataset element_spec=(TensorSpec(shape=(None, 256), dtype=tf.float64, name=None), TensorSpec(shape=(None, 128, 128, 1), dtype=tf.float64, name=None))>


## Training

In [ ]:
tf.keras.backend.clear_session()
flow_recon_net = model.VAE(input_shape= (128,128,1), latent_shape=(4,4,4))
flow_recon_net.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00005, beta_1 = 0.9, beta_2 = 0.999))
flow_recon_net.fit(dataset, epochs = 500, shuffle = True)

In [4]:
# Save model weight
flow_recon_net.encoder.save_weights('encoder.h5')
flow_recon_net.decoder.save_weights('decoder.h5')